<h1> Medical Differential Diagnosis with Mistral 7B RAG vs. BioMistral 7B </h1>
<h3> Comparing RAG-Enhanced General Models vs Domain-Specific Models </h3>

In [1]:
!pip install langchain_community transformers openpyxl sentence_transformers faiss-cpu pypdf hf_xet


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


<h3> Configuration </h3>

In [2]:
import pandas as pd
import numpy as np
import torch
import os
import json
from pathlib import Path
import warnings
import glob
warnings.filterwarnings('ignore')

# For RAG purpose
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

# Pipeline - using smaller alternatives for portfolio demo
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, 
    pipeline, BitsAndBytesConfig
)

In [3]:
# Configuration - adjustable

class Config:
    # Use smaller models for portfolio demo that are able to run locally
    RAG_MODEL = "microsoft/DialoGPT-medium"  # Naive LLMs, dummies
    BIO_MODEL = "dmis-lab/biobert-base-cased-v1.1"  # Bio domain LLMs, dummies
    
    # For actual implementation, these would be:
    # RAG_MODEL = "mistralai/Mistral-7B-Instruct-v0.1" 
    # BIO_MODEL = "BioMistral/BioMistral-7B"
    # In production, use API from HuggingFace: token required
    
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # dummies
    
    PROMPT = "What are the top 10 most likely differential diagnoses? List them in order of likelihood:"

    CURRENT_DIR = Path.cwd()
    DATA_DIR = Path("./data")
    CASES_FILE = DATA_DIR / "sample_cases.xlsx"
    VECTOR_DB_DIR = DATA_DIR / "vector_db"
    RESULTS_DIR = DATA_DIR / "results"
    
    # PDF processing settings, adjustable depends on the PDF characteristics
    CHUNK_SIZE = 1000
    CHUNK_OVERLAP = 200

    MAX_LENGTH = 1024 # Setting context's length
    MAX_NEW_TOKENS = 200 # Control output length
    BATCH_SIZE = 4

config = Config()

In [4]:
# Create directories
for dir_path in [config.DATA_DIR, config.VECTOR_DB_DIR, config.RESULTS_DIR]:
    dir_path.mkdir(exist_ok=True)

print(f"Configuration loaded. Current directory: {config.CURRENT_DIR}")
print(f"Data directory: {config.DATA_DIR}")

Configuration loaded. Current directory: d:\Data Gita\Documents\AI for Health - Stockholm University\8. NLP\Project\ddx
Data directory: data


<h3> Define RAG Class

In [5]:
class MedicalRAGSystem:
    """RAG system for medical differential diagnosis using PDF documents"""
    
    def __init__(self, config): # Initialization
        self.config = config
        self.embeddings = None
        self.vector_store = None
        self.documents = []
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=config.CHUNK_SIZE,
            chunk_overlap=config.CHUNK_OVERLAP,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
        )
        
    def load_embeddings(self):
        """Load embedding model"""
        print("Loading embedding model...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.config.EMBEDDING_MODEL,
            model_kwargs={'device': 'cpu'}  # Use CPU for compatibility, real app use GPU
        )
        print("Embeddings loaded successfully!")
        
    def find_pdf_files(self):
        """Find all PDF files in the current directory, assuming all PDF files are RAG resources"""
        pdf_files = list(self.config.CURRENT_DIR.glob("*.pdf"))
        if not pdf_files:
            print(f"No PDF files found in {self.config.CURRENT_DIR}")
            print("Please place your medical PDF documents in the same folder as this script.")
        else:
            print(f"Found {len(pdf_files)} PDF files:")
            for pdf_file in pdf_files:
                print(f"  - {pdf_file.name}")
        return pdf_files
    
    def load_pdf_documents(self):
        """Load and process PDF documents from the current directory"""
        pdf_files = self.find_pdf_files()
        
        if not pdf_files:
            print("WARNING: No PDF files found. Creating fallback sample documents...")
            self.create_sample_medical_knowledge()
            return
        
        print("Loading PDF documents...")
        all_documents = []
        
        for pdf_file in pdf_files:
            try:
                print(f"Processing: {pdf_file.name}")
                
                # Load PDF
                loader = PyPDFLoader(str(pdf_file))
                pages = loader.load()
                
                # Split into chunks
                chunks = self.text_splitter.split_documents(pages)
                
                # Add source metadata
                for chunk in chunks:
                    chunk.metadata.update({
                        'source_file': pdf_file.name,
                        'file_path': str(pdf_file),
                        'total_pages': len(pages)
                    })
                
                all_documents.extend(chunks)
                print(f"Loaded {len(chunks)} chunks from {pdf_file.name}")
                
            except Exception as e:
                print(f"Error processing {pdf_file.name}: {e}")
                continue
        
        if all_documents:
            self.documents = all_documents
            print(f"Successfully loaded {len(all_documents)} document chunks from {len(pdf_files)} PDF files")
        else:
            print("ERROR: Failed to load any PDF documents. Creating fallback documents...") # Fail-safe mechanism
            self.create_sample_medical_knowledge()

        
    def create_sample_medical_knowledge(self): # For demonstration purposes
        """Mock-up Medical Documents: as fallback if no PDFs found
        This is used for the fail-safe mechanism. Other mechanism is also possible, for example: looking at online references"""
        
        print("Creating fallback sample medical documents...")
        sample_docs = [
            {
                "title": "Acute Myocardial Infarction",
                "content": """Acute myocardial infarction (AMI) is characterized by chest pain, often radiating to the left arm, jaw, or back. Key features include ST elevation on ECG, elevated cardiac enzymes (troponin, CK-MB), and typical presentation during or after physical exertion. Risk factors include hypertension, diabetes, smoking, and family history. Immediate treatment includes aspirin, nitroglycerin, and emergency catheterization."""
            },
            {
                "title": "Bacterial Meningitis",
                "content": """Bacterial meningitis presents with classic triad of fever, headache, and neck stiffness. Positive Kernig's and Brudzinski's signs are pathognomonic. CSF analysis shows elevated WBC with neutrophil predominance, low glucose (<40% of serum), and elevated protein. Common organisms include Streptococcus pneumoniae, Neisseria meningitidis, and Haemophilus influenzae. Emergency treatment with broad-spectrum antibiotics is crucial."""
            },
            {
                "title": "Diabetic Ketoacidosis",
                "content": """DKA is characterized by hyperglycemia (>250 mg/dL), ketosis, and metabolic acidosis. Classic symptoms include polyuria, polydipsia, weight loss, and fruity breath odor. Laboratory findings show positive urine ketones, low bicarbonate, and elevated anion gap. Treatment involves IV fluids, insulin therapy, and electrolyte correction. Common precipitants include infection, medication noncompliance, and new-onset diabetes."""
            },
            {
                "title": "Pregnancy Complications",
                "content": """Abortion complications in pregnancy include threatened, inevitable, incomplete, and complete abortion. Inevitable abortion presents with open cervix and cramping pain. Incomplete abortion shows retained tissue with continued bleeding. Ultrasound helps differentiate between types. Placental abruption presents with severe abdominal pain and may show concealed hemorrhage."""
            },
            {
                "title": "Pediatric Infections",
                "content": """Streptococcal pharyngitis in children presents with fever, sore throat, tonsillar exudate, and cervical lymphadenopathy. Centor criteria help distinguish from viral causes. Rapid strep test or throat culture confirms diagnosis. Treatment includes penicillin or amoxicillin. Complications may include rheumatic fever if untreated."""
            }
        ]

        documents = []
        for doc in sample_docs:
            documents.append(Document(
                page_content=doc["content"],
                metadata={"title": doc["title"],
                    "source_file": "fallback_sample",
                    "file_path": "internal_sample"}
            ))
        
        self.documents = documents
        print(f"Created {len(documents)} fallback medical documents")
        
    def build_vector_store(self):
        """Build FAISS vector store from documents"""
        if not self.embeddings:
            self.load_embeddings()
            
        if not self.documents:
            self.load_pdf_documents()
        
        print("Building vector store...")
        self.vector_store = FAISS.from_documents(
            self.documents,
            self.embeddings
        )
        
        # Save vector store
        self.vector_store.save_local(str(self.config.VECTOR_DB_DIR))
        print(f"Vector store saved to {self.config.VECTOR_DB_DIR}")
        
        # Save document metadata for reference
        metadata_file = self.config.DATA_DIR / "document_metadata.json"
        metadata = []
        for doc in self.documents:
            metadata.append({
                "content_preview": doc.page_content[:200] + "...",
                "metadata": doc.metadata
            })
        
        with open(metadata_file, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
        print(f"Document metadata saved to {metadata_file}")

    def retrieve_context(self, query, k=3, max_context_length=2000):
        """Retrieve relevant context for a query"""
        if not self.vector_store:
            print("Loading vector store...")
            self.load_embeddings()
            try:
                self.vector_store = FAISS.load_local(
                    str(self.config.VECTOR_DB_DIR), 
                    self.embeddings
            )
            except Exception as e:
                print(f"Error loading vector store: {e}")
                print("Rebuilding vector store...")
                self.build_vector_store()
        

        docs = self.vector_store.similarity_search(query, k=k)
        sources = [doc.metadata.get("source_file", "Unknown") for doc in docs]
        
        # Truncate context to prevent token limit issues
        context_parts = []
        current_length = 0
        
        for doc in docs:
            content = doc.page_content
            if current_length + len(content) <= max_context_length:
                context_parts.append(content)
                current_length += len(content)
            else:
                # Add partial content if there's room
                remaining = max_context_length - current_length
                if remaining > 100:  # Only add if meaningful amount remains
                    context_parts.append(content[:remaining] + "...")
                break
        
        context = "\n\n".join(context_parts)
        return context, sources

    
    def get_document_statistics(self):
        """Get statistics about loaded documents"""
        if not self.documents:
            return {"error": "No documents loaded"}
        
        stats = {
            "total_chunks": len(self.documents),
            "source_files": {},
            "average_chunk_length": np.mean([len(doc.page_content) for doc in self.documents]),
            "total_content_length": sum([len(doc.page_content) for doc in self.documents])
        }
        
        # Count chunks per source file
        for doc in self.documents:
            source = doc.metadata.get("source_file", "Unknown")
            if source not in stats["source_files"]:
                stats["source_files"][source] = 0
            stats["source_files"][source] += 1
        
        return stats


In [6]:
# Initialize RAG system
print("\n" + "=" * 60)
print("INITIALIZING RAG SYSTEM")
print("=" * 60)
rag_system = MedicalRAGSystem(config)
rag_system.build_vector_store()


INITIALIZING RAG SYSTEM
Loading embedding model...


C:\Users\ignasius.dwi\AppData\Local\Temp\ipykernel_12272\2457445398.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


Embeddings loaded successfully!
No PDF files found in d:\Data Gita\Documents\AI for Health - Stockholm University\8. NLP\Project\ddx
Please place your medical PDF documents in the same folder as this script.
Creating fallback sample medical documents...
Created 5 fallback medical documents
Building vector store...
Vector store saved to data\vector_db
Document metadata saved to data\document_metadata.json


In [7]:
# Display document statistics
print("\n" + "=" * 60)
print("DOCUMENT STATISTICS")
print("=" * 60)
stats = rag_system.get_document_statistics()
if "error" not in stats:
    print(f"Total document chunks: {stats['total_chunks']}")
    print(f"Average chunk length: {stats['average_chunk_length']:.0f} characters")
    print(f"Total content length: {stats['total_content_length']:,} characters")
    print(f"Source files:")
    for source, count in stats['source_files'].items():
        print(f"  - {source}: {count} chunks")
else:
    print(stats["error"])


DOCUMENT STATISTICS
Total document chunks: 5
Average chunk length: 393 characters
Total content length: 1,966 characters
Source files:
  - fallback_sample: 5 chunks


In [8]:
# Test RAG retrieval, see if it's working
print("\n" + "=" * 60)
print("TESTING RAG RETRIEVAL")
print("=" * 60)
test_query = "patient with chest pain and ST elevation"
context, sources = rag_system.retrieve_context(test_query)
print(f"Query: '{test_query}'")
print(f"Sources: {sources}")
print(f"Context preview: {context[:300]}...")


TESTING RAG RETRIEVAL
Query: 'patient with chest pain and ST elevation'
Sources: ['fallback_sample', 'fallback_sample', 'fallback_sample']
Context preview: Acute myocardial infarction (AMI) is characterized by chest pain, often radiating to the left arm, jaw, or back. Key features include ST elevation on ECG, elevated cardiac enzymes (troponin, CK-MB), and typical presentation during or after physical exertion. Risk factors include hypertension, diabet...


<h3>Load the model</h3>

In [9]:
class MedicalDiagnosisModels:
    """Wrapper for both RAG-enhanced and Bio-specialized models"""
    
    def __init__(self, config, rag_system):
        self.config = config
        self.rag_system = rag_system
        self.rag_pipeline = None
        self.bio_pipeline = None
        
    def load_rag_model(self):
        """Load RAG-enhanced general model"""
        print(f"Loading RAG model: {self.config.RAG_MODEL}")
        
        # For portfolio demo - using smaller model
        # In production, this would be Mistral-7B with quantization
        try:
            tokenizer = AutoTokenizer.from_pretrained(self.config.RAG_MODEL, padding_side='left')
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token
                
            model = AutoModelForCausalLM.from_pretrained(
                self.config.RAG_MODEL,
                torch_dtype=torch.float32,  # CPU compatible, adjust for GPU implementation
                low_cpu_mem_usage=True
            )
            
            self.rag_pipeline = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=self.config.MAX_NEW_TOKENS,
                do_sample=True,
                temperature=0.7,
                truncation=True,
                pad_token_id=tokenizer.eos_token_id
            )
            print("RAG model loaded successfully!")
            
        except Exception as e:
            print(f"Error loading RAG model: {e}")
            print("Using mock RAG model for demonstration...")
            self.rag_pipeline = self._create_mock_rag_pipeline()
    
    def load_bio_model(self):
        """Load bio-specialized model"""
        print(f"Loading Bio model: {self.config.BIO_MODEL}")
        
        # For portfolio demo - using mock since BioMistral-7B is too large
        print("Using mock Bio model for demonstration...")
        self.bio_pipeline = self._create_mock_bio_pipeline()
    
    def _create_mock_rag_pipeline(self):
        """For demo purposes, this is safe-mechanism for fallback mock function. Another option is to show the error code and stop the process.
        This will activate if model loading fails, API limits are hit, or hardware constraints."""
        def mock_rag(prompt, **kwargs):
            # Simulate RAG-enhanced responses based on retrieved context
            if "chest pain" in prompt.lower() and "st elevation" in prompt.lower():
                return [{"generated_text": prompt + "\n\nBased on the retrieved medical literature, the top 10 differential diagnoses are:\n1. Acute ST-elevation myocardial infarction (STEMI)\n2. Non-ST elevation myocardial infarction (NSTEMI)\n3. Unstable angina pectoris\n4. Acute pericarditis\n5. Aortic dissection\n6. Pulmonary embolism\n7. Pneumothorax\n8. Esophageal rupture\n9. Gastroesophageal reflux disease\n10. Musculoskeletal chest pain"}]
            elif "fever" in prompt.lower() and "neck stiffness" in prompt.lower():
                return [{"generated_text": prompt + "\n\nBased on retrieved medical knowledge, the differential diagnoses include:\n1. Bacterial meningitis\n2. Viral meningitis\n3. Subarachnoid hemorrhage\n4. Brain abscess\n5. Encephalitis\n6. Migraine headache\n7. Tension headache\n8. Sinusitis\n9. Temporal arteritis\n10. Drug-induced meningitis"}]
            else:
                return [{"generated_text": prompt + "\n\n[RAG Model Response - Context-enhanced diagnosis based on retrieved medical literature]"}]
        return mock_rag
    
    def _create_mock_bio_pipeline(self):
        """For demo purposes, this is safe-mechanism for fallback mock function. Another option is to show the error code and stop the process.
        This will activate if model loading fails, API limits are hit, or hardware constraints."""
        def mock_bio(prompt, **kwargs):
            # Simulate bio-specialized responses
            if "chest pain" in prompt.lower():
                return [{"generated_text": prompt + "\n\nBio-specialized analysis:\n1. Acute myocardial infarction\n2. Unstable angina\n3. Aortic dissection\n4. Pulmonary embolism\n5. Pneumothorax\n6. Pericarditis\n7. Esophageal disorders\n8. Musculoskeletal pain\n9. Anxiety disorder\n10. Gastric reflux"}]
            elif "fever" in prompt.lower() and "headache" in prompt.lower():
                return [{"generated_text": prompt + "\n\nBio-specialized differential:\n1. Bacterial meningitis\n2. Viral meningitis\n3. Encephalitis\n4. Subarachnoid hemorrhage\n5. Brain tumor\n6. Migraine\n7. Cluster headache\n8. Sinusitis\n9. Hypertensive emergency\n10. Carbon monoxide poisoning"}]
            else:
                return [{"generated_text": prompt + "\n\n[Bio-Specialized Model Response - Domain-specific medical analysis]"}]
        return mock_bio
    
    def generate_rag_diagnosis(self, case_description):
        """Generate diagnosis using RAG-enhanced model"""
        if not self.rag_pipeline:
            self.load_rag_model()
        
        # Retrieve relevant context
        context, sources = self.rag_system.retrieve_context(case_description, max_context_length=1500)
        
        # Create RAG prompt
        prompt = f"""Context: {context[:1500]}

Case: {case_description[:500]}

{self.config.PROMPT}"""
        
        try:
            # Check prompt length
            tokenizer = self.rag_pipeline.tokenizer
            tokens = tokenizer.encode(prompt)
            
            if len(tokens) > 800:  # Leave room for generation
                print(f"Warning: Prompt too long ({len(tokens)} tokens), truncating...")
                # Truncate context more aggressively
                context = context[:800]

                #Shorter the prompt so it can fit the context window
                prompt = f"""Medical context: {context}

Case: {case_description[:300]} 

{self.config.PROMPT}:"""
            
            response = self.rag_pipeline(
                prompt, 
                max_new_tokens=self.config.MAX_NEW_TOKENS,
                do_sample=True,
                temperature=0.7,
                truncation=True
            )
            
            diagnosis = response[0]["generated_text"][len(prompt):].strip()
            return {
                "diagnosis": diagnosis,
                "sources": sources,
                "method": "RAG-Enhanced",
                "prompt_tokens": len(tokens) if 'tokens' in locals() else 0
            }
        except Exception as e:
            print(f"RAG generation error: {e}")
            return {
                "diagnosis": f"Error: {str(e)[:100]}...",
                "sources": sources,
                "method": "RAG-Enhanced",
                "prompt_tokens": 0
            }
        
    def generate_bio_diagnosis(self, case_description):
        """Generate diagnosis using bio-specialized model"""
        if not self.bio_pipeline:
            self.load_bio_model()
        
        truncated_case = case_description[:400] # Prevent token issues
        prompt = f"""Case: {truncated_case}

{self.config.PROMPT}"""
        
        try:
            response = self.bio_pipeline(
                prompt, 
                max_new_tokens=self.config.MAX_NEW_TOKENS,
                truncation=True
            )
            diagnosis = response[0]["generated_text"][len(prompt):].strip()
            return {
                "diagnosis": diagnosis,
                "sources": ["Bio-specialized training data"],
                "method": "Bio-Specialized"
            }
        except Exception as e:
            return {
                "diagnosis": f"Error: {str(e)[:100]}...",
                "sources": ["Error"],
                "method": "Bio-Specialized"
            }


<h3> Prepare the Dataset </h3>

In [10]:
#Mock-up Dataset: the production can implement file upload from .xlsx or .csv / API connection with EHS.

sample_cases = [
    {
        "doi": "DOI_001",
        "description": "45-year-old male presents with acute chest pain radiating to left arm, diaphoresis, and nausea. Pain started 2 hours ago during physical activity. Patient has history of hypertension and smoking. Vital signs: BP 160/90, HR 110, RR 20. ECG shows ST elevation in leads II, III, aVF.",
        "ground_truth": ["Acute myocardial infarction", "Unstable angina", "Aortic dissection"]
    },
    {
        "doi": "DOI_002", 
        "description": "28-year-old female with 3-day history of fever (39°C), headache, neck stiffness, and photophobia. No recent travel. Physical exam reveals positive Kernig's sign. CSF analysis shows elevated WBC count with neutrophil predominance, low glucose, high protein.",
        "ground_truth": ["Bacterial meningitis", "Viral meningitis", "Subarachnoid hemorrhage"]
    },
    {
        "doi": "DOI_003",
        "description": "65-year-old diabetic patient presents with polyuria, polydipsia, weight loss, and blurred vision over 2 weeks. Random glucose 450 mg/dL, ketones positive in urine. Patient appears dehydrated with fruity breath odor.",
        "ground_truth": ["Diabetic ketoacidosis", "Hyperosmolar hyperglycemic state", "Type 1 diabetes onset"]
    },
    {
        "doi": "DOI_004",
        "description": "32-year-old pregnant woman at 20 weeks gestation presents with sudden onset severe abdominal pain and vaginal bleeding. Pain is cramping in nature. Vital signs stable. Pelvic exam shows open cervix with tissue passage.",
        "ground_truth": ["Incomplete abortion", "Inevitable abortion", "Placental abruption"]
    },
    {
        "doi": "DOI_005",
        "description": "8-year-old child with 2-day history of high fever, sore throat, and difficulty swallowing. Physical exam reveals enlarged, erythematous tonsils with white exudate, tender cervical lymphadenopathy, and petechiae on soft palate.",
        "ground_truth": ["Streptococcal pharyngitis", "Viral pharyngitis", "Infectious mononucleosis"]
    }
]
df_cases = pd.DataFrame(sample_cases)
df_cases.to_excel(config.CASES_FILE, index=False)
print(f"Created sample cases file: {config.CASES_FILE}")

Created sample cases file: data\sample_cases.xlsx


<h3>Processing</h3>

In [11]:
# Initialize models
print("\n" + "=" * 60)
print("INITIALIZING MEDICAL DIAGNOSIS MODELS")
print("=" * 60)
diagnosis_models = MedicalDiagnosisModels(config, rag_system)

def process_all_cases(cases_df, models, output_file=None):
    """
    Process all cases efficiently by running all cases through one model first,
    then all cases through the second model. This avoids expensive model switching.
    """
    results = []
    
    print("Phase 1: Processing all cases with RAG-Enhanced Model")
    print("=" * 60)
    
    # Load RAG model once
    models.load_rag_model()
    
    # Process all cases with RAG model
    for idx, row in cases_df.iterrows():
        case_id = row['doi']
        description = row['description']
        
        print(f"Processing {case_id} with RAG model...")
        
        rag_result = models.generate_rag_diagnosis(description)
        
        result = {
            'doi': case_id,
            'description': description,
            'ground_truth': row.get('ground_truth', []),
            'rag_diagnosis': rag_result['diagnosis'],
            'rag_sources': rag_result['sources'],
            'bio_diagnosis': None,  # Will be filled later
            'bio_sources': None
        }
        
        results.append(result)
        print(f"DONE: {case_id} RAG processing complete")
    
    # Clean up RAG model to free memory
    print("\nCleaning up RAG model...")
    models.rag_pipeline = None
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    print("\nPhase 2: Processing all cases with Bio-Specialized Model")
    print("=" * 60)
    
    # Load Bio model
    models.load_bio_model()
    
    # Process all cases with Bio model
    for idx, result in enumerate(results):
        case_id = result['doi']
        description = result['description']
        
        print(f"Processing {case_id} with Bio model...")
        
        bio_result = models.generate_bio_diagnosis(description)
        
        result['bio_diagnosis'] = bio_result['diagnosis']
        result['bio_sources'] = bio_result['sources']
        
        print(f"DONE: {case_id} Bio processing complete")
    
    # Save results
    if output_file:
        results_df = pd.DataFrame(results)
        results_df.to_excel(output_file, index=False)
        print(f"\nResults saved to: {output_file}")
    
    return results


INITIALIZING MEDICAL DIAGNOSIS MODELS


In [12]:
# Load sample cases
cases_df = pd.read_excel(config.CASES_FILE)
print(f"Loaded {len(cases_df)} cases for processing")

Loaded 5 cases for processing


In [13]:
# Process all cases
output_file = config.RESULTS_DIR / "diagnosis_comparison_results.xlsx"
results = process_all_cases(cases_df, diagnosis_models, output_file)


Phase 1: Processing all cases with RAG-Enhanced Model
Loading RAG model: microsoft/DialoGPT-medium


Device set to use cpu


RAG model loaded successfully!
Processing DOI_001 with RAG model...
DONE: DOI_001 RAG processing complete
Processing DOI_002 with RAG model...
DONE: DOI_002 RAG processing complete
Processing DOI_003 with RAG model...
DONE: DOI_003 RAG processing complete
Processing DOI_004 with RAG model...
DONE: DOI_004 RAG processing complete
Processing DOI_005 with RAG model...
DONE: DOI_005 RAG processing complete

Cleaning up RAG model...

Phase 2: Processing all cases with Bio-Specialized Model
Loading Bio model: dmis-lab/biobert-base-cased-v1.1
Using mock Bio model for demonstration...
Processing DOI_001 with Bio model...
DONE: DOI_001 Bio processing complete
Processing DOI_002 with Bio model...
DONE: DOI_002 Bio processing complete
Processing DOI_003 with Bio model...
DONE: DOI_003 Bio processing complete
Processing DOI_004 with Bio model...
DONE: DOI_004 Bio processing complete
Processing DOI_005 with Bio model...
DONE: DOI_005 Bio processing complete

Results saved to: data\results\diagnosis

<h3>Result</h3>

In [14]:
# Display results
print("\n" + "=" * 80)
print("DIAGNOSIS COMPARISON RESULTS")
print("=" * 80)

for i, result in enumerate(results[:2]):  # Show first 2 results
    print(f"DOI {i+1}: {result['doi']}")
    print("-" * 40)
    print(f"Description: {result['description'][:100]}...")
    print(f"Ground Truth: {result['ground_truth']}")
    
    print(f"\nRAG-Enhanced Model:")
    print(f"Sources: {result['rag_sources']}")
    print(f"Diagnosis: {result['rag_diagnosis'][:200]}...")
    
    print(f"\nBio-Specialized Model:")
    print(f"Sources: {result['bio_sources']}")
    print(f"Diagnosis: {result['bio_diagnosis'][:200]}...")
    print("\n" + "-" * 80)



DIAGNOSIS COMPARISON RESULTS
DOI 1: DOI_001
----------------------------------------
Description: 45-year-old male presents with acute chest pain radiating to left arm, diaphoresis, and nausea. Pain...
Ground Truth: ['Acute myocardial infarction', 'Unstable angina', 'Aortic dissection']

RAG-Enhanced Model:
Sources: ['fallback_sample', 'fallback_sample', 'fallback_sample']
Diagnosis: ...

Bio-Specialized Model:
Sources: ['Bio-specialized training data']
Diagnosis: Bio-specialized analysis:
1. Acute myocardial infarction
2. Unstable angina
3. Aortic dissection
4. Pulmonary embolism
5. Pneumothorax
6. Pericarditis
7. Esophageal disorders
8. Musculoskeletal pain
9...

--------------------------------------------------------------------------------
DOI 2: DOI_002
----------------------------------------
Description: 28-year-old female with 3-day history of fever (39°C), headache, neck stiffness, and photophobia. No...
Ground Truth: ['Bacterial meningitis', 'Viral meningitis', 'Subarachnoi